In [39]:
import pickle
from pandas import DataFrame


try:
    with open("data/bloomData.pkl", "rb") as dataPickle:
        fullData: DataFrame = pickle.load(dataPickle, fix_imports=True)
except:
    fullData = DataFrame()

## Filter to important entries

In [40]:
data = fullData[
    [
        "communityvisibilitystate",
        "personastate",
        "timecreated",
        "CommunityBanned",
        "NumberOfVACBans",
        "DaysSinceLastBan",
        "NumberOfGameBans",
        "EconomyBan",
        "friendsList",
        "ownedGames",
        "groups",
        "recentlyPlayed",
        "commentpermission",
        "loccountrycode",
        "avatar",
    ]
]

data.replace({"EconomyBan":{"banned": True, "none": False}}, inplace=True)



/tmp/ipykernel_113432/1340769248.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"EconomyBan":{"banned": True, "none": False}}, inplace=True)
/tmp/ipykernel_113432/1340769248.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace({"EconomyBan":{"banned": True, "none": False}}, inplace=True)


In [41]:
from datetime import date, datetime, timedelta

from pandas import Series, Timedelta


DEFAULT_AVATAR = (
    "https://avatars.steamstatic.com/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg"
)

data["MisdemenourCount"] = (
    data["NumberOfVACBans"]
    + data["NumberOfGameBans"]
    + data["CommunityBanned"].apply(lambda b: 1 if b else 0)
    + data["EconomyBan"].apply(lambda b: 1 if b else 0)
)


# Doesn't account for people who doesn't publicize the info
data["ActiveAfterBan"] = data["recentlyPlayed"].apply(
    lambda d: d.get("total_count", 0) > 0
) & (data["NumberOfVACBans"] > 0)

data["HasDefaultAvatar"] = data["avatar"] == DEFAULT_AVATAR
data["WeWereBadButNowWereGood"] = data["DaysSinceLastBan"] > 365

currentTimestamp = datetime.now()
data["LastBanTime"] = data["DaysSinceLastBan"].apply(
    lambda d: (currentTimestamp - timedelta(d)).timestamp() if d > 0 else None
)
data["EstimatedFirstBan"] = (
    data["DaysSinceLastBan"] * (data["NumberOfVACBans"] + data["NumberOfGameBans"])
).apply(lambda d: (currentTimestamp - timedelta(d)).timestamp() if d > 0 else None)

/tmp/ipykernel_113432/2469785220.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["MisdemenourCount"] = (
/tmp/ipykernel_113432/2469785220.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["ActiveAfterBan"] = data["recentlyPlayed"].apply(
/tmp/ipykernel_113432/2469785220.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan